# 3. Renaming and processing metadata files

In [8]:
library(dplyr)
library(stringr)
library(tidyr)

In [12]:
# Process metadata with number of reads
## Load the metadata file
setwd("/home/strawberry/Documents/Collaborations/sinusite_edwin/data") # always start setting your working dir 
metadata <- read.table("metadata_nreads.tsv", header = TRUE, sep = "\t", stringsAsFactors = FALSE)
head(metadata)

,ID_Sample,Filename,nreads,Patient,Timepoint
,<chr>,<chr>,<int>,<chr>,<chr>
1,0001967K_T1,0001967K_T1.fastq,1061,0001967K,T1
2,0001967K_T3,0001967K_T3.fastq,20860,0001967K,T3
3,0001967K_T6,0001967K_T6.fastq,48290,0001967K,T6
4,0018303G_T0,0018303G_T0.fastq,25719,0018303G,T0
5,0088303G_T0,0088303G_T0.fastq,9051,0088303G,T0
6,0088303G_T1,0088303G_T1.fastq,11707,0088303G,T1


#### OBS: 
##### I've split the metadata (clinical data, snot22 scores, etc) into two: one that contains the general information 
##### regarding the patients on this study and a second one with those specific to the samples, 
##### i.e. clinical parameters measured in each time point (snot, corticoid and antibiotic use, etc)

In [14]:
# Process metadata with patient general information
## Load metadata_patients file in R 
metadata_patients <- read.table("metadata_patients.tsv", header = TRUE, sep = "\t", stringsAsFactors = FALSE)
head(metadata_patients)

## Merge metadata (with the number of reads and files names) and metadata_patients by "Patient"
merged_metadata <- merge(metadata, metadata_patients, by = "Patient", all.x = TRUE)
head(merged_metadata)

,Patient,Sex,Age,Atopic_Symptoms,Asthma,NSAID_Intolerance,Total_IgE,Blood_Eosinophilia,Tissue_Eosinophilia,Final_LundMackay
,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
1,0207720E,1,56.6,0,0,0,16.6,200,20,14
2,1560534I,1,45.9,1,0,1,123.0,600,100,22
3,0358613K,2,26.2,1,0,0,64.0,600,15,24
4,1578804D,1,47.0,1,1,1,50.4,1600,180,24
5,1532317F,2,46.7,1,0,0,54.1,300,45,24
6,1377897D,1,44.8,0,1,1,171.0,400,240,24


,Patient,ID_Sample,Filename,nreads,Timepoint,Sex,Age,Atopic_Symptoms,Asthma,NSAID_Intolerance,Total_IgE,Blood_Eosinophilia,Tissue_Eosinophilia,Final_LundMackay
,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
1,0001967K,0001967K_T6,0001967K_T6.fastq,48290,T6,1,58.7,1,1,0,38,100,10,15
2,0001967K,0001967K_T1,0001967K_T1.fastq,1061,T1,1,58.7,1,1,0,38,100,10,15
3,0001967K,0001967K_T3,0001967K_T3.fastq,20860,T3,1,58.7,1,1,0,38,100,10,15
4,0018303G,0018303G_T0,0018303G_T0.fastq,25719,T0,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,0088303G,0088303G_T0,0088303G_T0.fastq,9051,T0,2,60.3,1,1,0,667,800,58,22
6,0088303G,0088303G_T3,0088303G_T3.fastq,3587,T3,2,60.3,1,1,0,667,800,58,22


In [16]:
## Remove patients with sequenced samples but without any metadata about them (i.e. NA in all columns from 6-14)
merged_metadata <- merged_metadata[rowSums(is.na(merged_metadata[, 6:14])) < 9, ] 

## Save metadata table
write.table(merged_metadata, "merged_metadata.tsv", sep = "\t", row.names = FALSE, quote = FALSE)
head(merged_metadata)
length(unique(merged_metadata$ID_Sample)) # couting how many files remain after this filtering step

,Patient,ID_Sample,Filename,nreads,Timepoint,Sex,Age,Atopic_Symptoms,Asthma,NSAID_Intolerance,Total_IgE,Blood_Eosinophilia,Tissue_Eosinophilia,Final_LundMackay
,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
1,0001967K,0001967K_T6,0001967K_T6.fastq,48290,T6,1,58.7,1,1,0,38,100,10,15
2,0001967K,0001967K_T1,0001967K_T1.fastq,1061,T1,1,58.7,1,1,0,38,100,10,15
3,0001967K,0001967K_T3,0001967K_T3.fastq,20860,T3,1,58.7,1,1,0,38,100,10,15
5,0088303G,0088303G_T0,0088303G_T0.fastq,9051,T0,2,60.3,1,1,0,667,800,58,22
6,0088303G,0088303G_T3,0088303G_T3.fastq,3587,T3,2,60.3,1,1,0,667,800,58,22
7,0088303G,0088303G_T1,0088303G_T1.fastq,11707,T1,2,60.3,1,1,0,667,800,58,22


[1] 160

In [17]:
# Process metadata with timepoint specific information
## Load metadata_timepoints file in R 
metadata_timepoints <- read.table("metadata_timepoints.tsv", header = TRUE, sep = "\t", stringsAsFactors = FALSE)
head(metadata_timepoints)

## Process metadata_timepoints: Append _T0, _T1, _T3, _T6, and _T12 to Patients codes --> new first column of a processed metadata_timepoints df 
### Define timepoints to append
timepoints <- c("_T0", "_T1", "_T3", "_T6", "_T12")
timepoints

,Patient,SNOT22_T0,SNOT22_T1,SNOT22_T3,SNOT22_T6,SNOT22_T12,SNOT22nasal_T0,SNOT22nasal_T1,SNOT22nasal_T3,SNOT22nasal_T6,⋯,ClinicCtrl_T12,AntUse_T1,AntUse_T3,AntUse_T6,AntUse_T12,Cortuse_T0,Cortuse_T1,Cortuse_T3,Cortuse_T6,Cortuse_T12
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0207720E,37,34,22,54,46,16,11,9,17,⋯,NC,0,0,0,0,0,0,0,0,0
2,1560534I,39,16,17,9,17,19,3,3,3,⋯,PC,1,1,0,1,1,1,0,0,0
3,0358613K,39,0,15,17,12,27,0,15,14,⋯,PC,1,0,0,0,0,1,0,1,1
4,1578804D,101,11,59,59,76,35,5,20,13,⋯,NC,1,0,1,0,0,0,0,0,0
5,1532317F,49,3,2,5,6,21,2,2,5,⋯,NC,1,0,0,0,0,1,0,0,0
6,1377897D,60,1,32,40,31,26,1,12,9,⋯,PC,1,0,0,0,0,1,0,0,0


[1] "_T0"  "_T1"  "_T3"  "_T6"  "_T12"

In [18]:
## Add a new column "AntUse_T0" filled with NA before "AntUse_T1"
metadata_timepoints <- metadata_timepoints %>%
  mutate(AntUse_T0 = NA) %>%
  relocate(AntUse_T0, .before = AntUse_T1)

## Fill missing values in ClinicCtrl_ columns with NA
metadata_timepoints <- metadata_timepoints %>%
  mutate(across(starts_with("ClinicCtrl_"), ~ ifelse(. == "", NA, .)))
    
head(metadata_timepoints) 

,Patient,SNOT22_T0,SNOT22_T1,SNOT22_T3,SNOT22_T6,SNOT22_T12,SNOT22nasal_T0,SNOT22nasal_T1,SNOT22nasal_T3,SNOT22nasal_T6,⋯,AntUse_T0,AntUse_T1,AntUse_T3,AntUse_T6,AntUse_T12,Cortuse_T0,Cortuse_T1,Cortuse_T3,Cortuse_T6,Cortuse_T12
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<lgl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0207720E,37,34,22,54,46,16,11,9,17,⋯,NA,0,0,0,0,0,0,0,0,0
2,1560534I,39,16,17,9,17,19,3,3,3,⋯,NA,1,1,0,1,1,1,0,0,0
3,0358613K,39,0,15,17,12,27,0,15,14,⋯,NA,1,0,0,0,0,1,0,1,1
4,1578804D,101,11,59,59,76,35,5,20,13,⋯,NA,1,0,1,0,0,0,0,0,0
5,1532317F,49,3,2,5,6,21,2,2,5,⋯,NA,1,0,0,0,0,1,0,0,0
6,1377897D,60,1,32,40,31,26,1,12,9,⋯,NA,1,0,0,0,0,1,0,0,0


In [19]:
## Repeat each patient ID for each timepoint
metadata_timepoints_modified <- data.frame(
  Patient_ID = rep(metadata_timepoints$Patient, each = length(timepoints)),  # Repeat patient IDs
  Timepoint = rep(timepoints, times = nrow(metadata_timepoints))  # Assign corresponding timepoints
)

## Append timepoints to patient IDs
metadata_timepoints_modified$Patient_ID <- paste0(metadata_timepoints_modified$Patient_ID, metadata_timepoints_modified$Timepoint)

## Remove the separate "Timepoint" column
metadata_timepoints_modified$Timepoint <- NULL

## Add columns from unique_new_colnames and fill with NA
unique_new_colnames <- unique(gsub("_T.*", "", colnames(metadata_timepoints)[-1]))  # Get unique column names
metadata_timepoints_modified[unique_new_colnames] <- NA  # Initialize new columns with NA

head(metadata_timepoints_modified)

,Patient_ID,SNOT22,SNOT22nasal,LK,ClinicCtrl,AntUse,Cortuse
,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,0207720E_T0,NA,NA,NA,NA,NA,NA
2,0207720E_T1,NA,NA,NA,NA,NA,NA
3,0207720E_T3,NA,NA,NA,NA,NA,NA
4,0207720E_T6,NA,NA,NA,NA,NA,NA
5,0207720E_T12,NA,NA,NA,NA,NA,NA
6,1560534I_T0,NA,NA,NA,NA,NA,NA


In [26]:
## Fill values by hand. I've tried using some codes to do so, but no success, so I downloaded the tables put them on excel and edited them
write.table(metadata_timepoints_modified, "metadata_timepoints_modified.tsv", sep = "\t", row.names = FALSE, quote = FALSE)
write.table(metadata_timepoints, "metadata_timepoints_processed.tsv", sep = "\t", row.names = FALSE, quote = FALSE)

In [20]:
## Import metadata_timepoints df back after processing by hand 
### (I transposed values individually to have a long format df)
metadata_timepoints_processed <- read.table("metadata_timepoints_processed.csv", header = TRUE, sep = "\t", stringsAsFactors = FALSE)
head(metadata_timepoints_processed)

,ID_Sample,SNOT22,SNOT22nasal,LK_Total,LK_Secret,ClinicCtrl,AntUse,Cortuse
,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>
1,0207720E_T0,37,16,9,2,NC,NA,0
2,0207720E_T1,34,11,1,1,C,0,0
3,0207720E_T3,22,9,2,0,C,0,0
4,0207720E_T6,54,17,2,2,PC,0,0
5,0207720E_T12,46,13,6,2,NC,0,0
6,1560534I_T0,39,19,5,1,NC,NA,1


In [21]:
# Merge metadata to a single df 
metadata_merged_full <- merge(metadata_timepoints_processed, merged_metadata, by = "ID_Sample", all.x = TRUE)
metadata_merged_full # lots of patients samples in specific timepoints with missing files

ID_Sample,SNOT22,SNOT22nasal,LK_Total,LK_Secret,ClinicCtrl,AntUse,Cortuse,Patient,Filename,⋯,Timepoint,Sex,Age,Atopic_Symptoms,Asthma,NSAID_Intolerance,Total_IgE,Blood_Eosinophilia,Tissue_Eosinophilia,Final_LundMackay
<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
0001967K_T0,95,26,11,4,NC,NA,0,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0001967K_T1,38,12,4,2,NC,1,1,0001967K,0001967K_T1.fastq,⋯,T1,1,58.7,1,1,0,38.0,100,10,15
0001967K_T12,NA,NA,NA,NA,NA,0,0,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0001967K_T3,96,26,10,4,NC,1,0,0001967K,0001967K_T3.fastq,⋯,T3,1,58.7,1,1,0,38.0,100,10,15
0001967K_T6,79,19,6,4,NC,1,1,0001967K,0001967K_T6.fastq,⋯,T6,1,58.7,1,1,0,38.0,100,10,15
0088303G_T0,102,37,8,2,NC,NA,0,0088303G,0088303G_T0.fastq,⋯,T0,2,60.3,1,1,0,667.0,800,58,22
0088303G_T1,32,7,4,1,NC,1,0,0088303G,0088303G_T1.fastq,⋯,T1,2,60.3,1,1,0,667.0,800,58,22
0088303G_T12,NA,NA,NA,NA,NA,0,0,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0088303G_T3,NA,NA,NA,NA,NA,0,0,0088303G,0088303G_T3.fastq,⋯,T3,2,60.3,1,1,0,667.0,800,58,22


In [22]:
# Count rows without a Filename
num_na_filename <- sum(is.na(metadata_merged_full$Filename)) # count
cat("Number of rows with NA in Filename:", num_na_filename, "\n") # display count

# Remove rows where Filename is NA
metadata_full_clean <- subset(metadata_merged_full, !is.na(Filename))
head(metadata_full_clean)
write.table(metadata_full_clean, "metadata_full_clean.tsv", sep = "\t", row.names = FALSE, quote = FALSE)

Number of rows with NA in Filename: 54 


,ID_Sample,SNOT22,SNOT22nasal,LK_Total,LK_Secret,ClinicCtrl,AntUse,Cortuse,Patient,Filename,⋯,Timepoint,Sex,Age,Atopic_Symptoms,Asthma,NSAID_Intolerance,Total_IgE,Blood_Eosinophilia,Tissue_Eosinophilia,Final_LundMackay
,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<int>
2,0001967K_T1,38,12,4,2,NC,1,1,0001967K,0001967K_T1.fastq,⋯,T1,1,58.7,1,1,0,38,100,10,15
4,0001967K_T3,96,26,10,4,NC,1,0,0001967K,0001967K_T3.fastq,⋯,T3,1,58.7,1,1,0,38,100,10,15
5,0001967K_T6,79,19,6,4,NC,1,1,0001967K,0001967K_T6.fastq,⋯,T6,1,58.7,1,1,0,38,100,10,15
6,0088303G_T0,102,37,8,2,NC,NA,0,0088303G,0088303G_T0.fastq,⋯,T0,2,60.3,1,1,0,667,800,58,22
7,0088303G_T1,32,7,4,1,NC,1,0,0088303G,0088303G_T1.fastq,⋯,T1,2,60.3,1,1,0,667,800,58,22
9,0088303G_T3,NA,NA,NA,NA,NA,0,0,0088303G,0088303G_T3.fastq,⋯,T3,2,60.3,1,1,0,667,800,58,22
